popular

In [ ]:
import pandas as pd

# Load the book data and ratings data into separate dataframes
book_data = pd.read_excel('books_1.Best_Books_Ever.csv.xlsx')
ratings_data = pd.read_csv('goodreads_interactions (1).csv')

# Merge the two dataframes on the ISBN column to combine book information and ratings information
book_ratings = pd.merge(book_data, ratings_data, on='book_id')

# Group the data by ISBN and compute the average rating and number of ratings for each book
grouped_data = book_ratings.groupby(['book_id']).agg({'rating': 'mean', 'numRatings': 'count'})

# Create a popularity score for each book based on the average rating and number of ratings
grouped_data['popularity_score'] = (grouped_data['rating'] * grouped_data['numRatings']) / (grouped_data['numRatings'] + 10)

# Sort the data by popularity score in descending order and print the top 10 books
grouped_data.sort_values(by=['popularity_score'], ascending=False, inplace=True)
print(grouped_data.head(10))

         rating  numRatings  popularity_score
book_id                                      
1408       4.85         238          4.654435
7351       4.79         321          4.645287
1000       4.59         844          4.536253
410        4.73         207          4.512028
21897      4.93         103          4.493717
19060      4.75         152          4.456790
1601       4.68         178          4.431064
5239       4.57         314          4.428951
759        4.48         487          4.389859
19964      4.75         108          4.347458


genres

In [ ]:
import pandas as pd

# load the books excel file
books_df = pd.read_excel('books_1.Best_Books_Ever.csv.xlsx')

# get user's favorite genres
user_fav_genres = input('Enter your favorite genres (separated by commas): ').split(',')

# use content-based filtering to filter books based on matching genres
genre_cols = ['genre 1', 'genre 2', 'genre 3', 'genre 4','genre 5']
matched_books_df = books_df[books_df[genre_cols].isin(user_fav_genres).any(axis=1)]
genre_counts = matched_books_df[genre_cols].apply(lambda x: sum(x.isin(user_fav_genres)), axis=1)
matched_books_df = matched_books_df[genre_counts >= 2].reset_index(drop=True)

# use collaborative filtering to filter out books already read and rated highly by the user
user_df = pd.read_csv('goodreads_interactions (1).csv')
user_ratings_df = user_df[user_df['Rating'] >= 4]  # consider only high ratings
user_rated_book_ids = user_ratings_df['book_id'].unique()
collab_filtered_books_df = matched_books_df[~matched_books_df['book_id'].isin(user_rated_book_ids)]

# calculate weighted score for the remaining books
C = books_df['rating'].mean()
m = books_df['numRatings'].quantile(0.9)
collab_filtered_books_df['weighted_score'] = (collab_filtered_books_df['numRatings'] / (collab_filtered_books_df['numRatings'] + m) * collab_filtered_books_df['rating']) + (m / (collab_filtered_books_df['numRatings'] + m) * C)

# sort the books by weighted score and display top 10 recommendations
collab_filtered_books_df = collab_filtered_books_df.sort_values('weighted_score', ascending=False).reset_index(drop=True)
print(f"Recommended books for {', '.join(user_fav_genres)}:\n")
for i, book in collab_filtered_books_df.head(10).iterrows():
    print(f"{i+1}. {book['book_id']} ({book['title']}) - {book['weighted_score']:.2f} weighted score, {book['likedPercent']}% liked")

Enter your favorite genres (separated by commas): Comedy,Romance,Classics
Recommended books for Comedy, Romance, Classics:

1. 4 (Pride and Prejudice) - 4.26 weighted score, 94.0% liked
2. 357 (The Complete Novels) - 4.24 weighted score, 97.0% liked
3. 16 (Memoirs of a Geisha) - 4.12 weighted score, 95.0% liked
4. 10183 (Carry On, Jeeves) - 4.09 weighted score, 98.0% liked
5. 70 (Little Women) - 4.09 weighted score, 93.0% liked
6. 15021 (The Inimitable Jeeves) - 4.08 weighted score, 98.0% liked
7. 1462 (Katherine) - 4.08 weighted score, 95.0% liked
8. 5855 (Leave It to Psmith) - 4.06 weighted score, 98.0% liked
9. 538 (Much Ado About Nothing) - 4.05 weighted score, 94.0% liked
10. 437 (The Scarlet Pimpernel) - 4.05 weighted score, 94.0% liked


<ipython-input-4-1260f294170d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collab_filtered_books_df['weighted_score'] = (collab_filtered_books_df['numRatings'] / (collab_filtered_books_df['numRatings'] + m) * collab_filtered_books_df['rating']) + (m / (collab_filtered_books_df['numRatings'] + m) * C)


CF auto

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, GRU, LSTM
from keras.models import Model
from keras import regularizers, optimizers
from keras.metrics import mean_absolute_error
from keras.metrics import MeanSquaredError
from sklearn.metrics import mean_squared_error

from keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, GRU, LSTM, SeparableConv1D

books_df = pd.read_excel('books_1.Best_Books_Ever.csv.xlsx')
users_df = pd.read_csv('goodreads_interactions (1).csv')

# Merge dataframes
merged_df = pd.merge(users_df, books_df, on='book_id')

# One-hot encode genre
genres = pd.get_dummies(merged_df['genre 1'])
merged_df = pd.concat([merged_df, genres], axis=1)

# Create user-item matrix
user_item_matrix = pd.pivot_table(merged_df, values='is_reviewed', index='user_id', columns='book_id')
user_item_matrix = user_item_matrix.fillna(0)

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(user_item_matrix)

# Define autoencoder model
input_layer = Input(shape=(user_item_matrix.shape[1],))
encoded = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(10e-5))(input_layer)
encoded = Dropout(0.5)(encoded)
encoded = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(10e-5))(encoded)
encoded = Dropout(0.5)(encoded)
encoded = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(10e-5))(encoded)
encoded = Dropout(0.5)(encoded)

encoded = encoded[:, :, np.newaxis]  # add new axis for channels
encoded = SeparableConv1D(64, 3, activation='relu', padding='same')(encoded)

#encoded = Conv1D(64, 3, activation='relu', padding='same')(encoded)
encoded = MaxPooling1D(2)(encoded)
encoded = GRU(32, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(encoded)
encoded = LSTM(32, activation='relu', dropout=0.2, recurrent_dropout=0.2)(encoded)

encoded = encoded[:, :, np.newaxis]  # add new axis for channels
encoded = SeparableConv1D(64, 3, activation='relu', padding='same')(encoded)

encoded = Conv1D(32, 5, activation='relu', padding='same')(encoded)
encoded = MaxPooling1D(2)(encoded)

encoded = GRU(32, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(encoded)
encoded = LSTM(32, activation='relu', dropout=0.2, recurrent_dropout=0.2)(encoded)


decoded = Dense(128, activation='relu')(encoded)
decoded = Dropout(0.5)(decoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dropout(0.5)(decoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dropout(0.5)(decoded)
decoded = Dense(user_item_matrix.shape[1], activation='sigmoid')(decoded)
autoencoder = Model(input_layer, decoded)

# Compile autoencoder model
optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
autoencoder.compile(optimizer=optimizer, loss='mean_squared_error')

# Train autoencoder
history = autoencoder.fit(user_item_matrix, user_item_matrix, epochs=10, batch_size=128, validation_split=0.2)



# Predict filtered matrix
filtered_matrix = autoencoder.predict(user_item_matrix)

filtered_matrix = np.nan_to_num(filtered_matrix)
filtered_cosine_sim = cosine_similarity(filtered_matrix)


import numpy as np

def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Calculate MSE, MAE, and MAPE
mse = mean_squared_error(user_item_matrix, filtered_matrix)
mae = mean_absolute_error(user_item_matrix, filtered_matrix)
mape = mean_absolute_percentage_error(user_item_matrix, filtered_matrix)
print("MSE:", mse)
#print("MAE:", mae)
#print("MAPE:", mape)

import numpy as np

mae_value = np.mean(mae.numpy())
print("MAE:", mae_value)

rmse = np.sqrt(mse)
print("RMSE:", rmse)



import random
def recommend_books(user_id, num_recommendations=50):
    user_row = user_item_matrix.loc[user_id]
    user_similarities = pd.Series(cosine_sim[user_id])
    top_users = user_similarities.sort_values(ascending=False)[1:num_recommendations+1].index
    top_users_filtered = filtered_matrix[top_users]
    weighted_ratings = np.dot(top_users_filtered, user_row) / np.sum(top_users_filtered, axis=1)
    books_read = user_row[user_row > 0].index
    books_unread = books_df[~books_df['book_id'].isin(books_read)]['book_id']
    recommended_books = [book_id for book_id in books_unread if book_id in books_df.index]
    recommended_books = random.sample(recommended_books, min(num_recommendations, len(recommended_books)))
    return books_df.loc[recommended_books]


    recommendations = recommend_books(user_id=500, num_recommendations=10)
    print(recommendations)


  # Define book recommendation function
    #def recommend_books(user_id, num_recommendations=50):
   # user_row = user_item_matrix.loc[user_id]
   # user_similarities = pd.Series(cosine_sim[user_id])
   # top_users = user_similarities.sort_values(ascending=False)[1:num_recommendations+1].index
   # top_users_filtered = filtered_matrix[top_users]
   # weighted_ratings = np.dot(top_users_filtered, user_row) / np.sum(top_users_filtered, axis=1)
   # top_books = weighted_ratings.argsort()[::-1][:num_recommendations]
   # return books_df.loc[top_books]



Epoch 1/10
13/13 [==============================] - 20s 880ms/step - loss: 0.3446 - val_loss: 0.2906
Epoch 2/10
13/13 [==============================] - 10s 775ms/step - loss: 0.2738 - val_loss: 0.2501
Epoch 3/10
13/13 [==============================] - 10s 785ms/step - loss: 0.1461 - val_loss: 0.0158
Epoch 4/10
13/13 [==============================] - 10s 785ms/step - loss: 0.0125 - val_loss: 0.0093
Epoch 5/10
13/13 [==============================] - 10s 763ms/step - loss: 0.0077 - val_loss: 0.0058
Epoch 6/10
13/13 [==============================] - 10s 717ms/step - loss: 0.0047 - val_loss: 0.0035
Epoch 7/10
13/13 [==============================] - 10s 783ms/step - loss: 0.0030 - val_loss: 0.0022
Epoch 8/10
13/13 [==============================] - 10s 788ms/step - loss: 0.0019 - val_loss: 0.0014
Epoch 9/10
13/13 [==============================] - 10s 787ms/step - loss: 0.0013 - val_loss: 9.7044e-04
Epoch 10/10
64/64 [==============================] - 6s 76ms/step
MSE: 0.00038079880927

<ipython-input-2-f2e46163ce55>:87: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
recommendations = recommend_books(user_id=500, num_recommendations=10)
print(recommendations)


       book_id                               bookId  \
1697      1775       2247142.The_Talented_Mr_Ripley   
8804      9704         6750.The_Broom_of_the_System   
14511    16438                   24910.World_s_Fair   
29694    39317  18485945-the-destiny-of-violet-luke   
6485      7038       23961.Marabou_Stork_Nightmares   
4965      5339     18667779-everything-leads-to-you   
2258      2373                 15768191-sweet-peril   
7138      7779                  30016.The_Naked_Sun   
30864    40924                    7884898-intrigues   
18953    23468        35197947-nekomonogatari-white   

                                  title                             series  \
1697            The Talented Mr. Ripley                          Ripley #1   
8804            The Broom of the System                                NaN   
14511                      World's Fair                                NaN   
29694      The Destiny of Violet & Luke                 The Coincidence #3   
6485

<ipython-input-3-a24bb1c3deb7>:113: RuntimeWarning: invalid value encountered in true_divide
  weighted_ratings = np.dot(top_users_filtered, user_row) / np.sum(top_users_filtered, axis=1)
